# call LLM

In [ ]:
from utils import call_llm, MODEL

prompt = f"""hello"""

print(call_llm(prompt, MODEL))

In [ ]:
from agent import get_ans
from utils import *

question = "What happened on this day in history? (UTC+0)"

get_ans(question, MODEL)

# Jina

In [ ]:
from utils import call_jina_search

res = call_jina_search("On what exact date did that London club Chelsea win the UEFA Champions League in that year before 2024?")

print(res)

# 拆分md文档

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter


with open("data.txt", "r", encoding="utf-8") as file:
    data = file.read()

# 指定要分割的标题
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

# 创建MarkdownHeaderTextSplitter实例
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)
md_header_splits = markdown_splitter.split_text(data)


# 打印分割后的结果
for i, split in enumerate(md_header_splits):
    print(f"Split {i + 1}:\n{split}\n")


# embedding

In [ ]:
# openai
from agent import client

response = client.embeddings.create(
    model="text-embedding-ada-002",
    input=["Hello world", "Another sentence"]
)

print(response.data)

In [ ]:
import ollama

res = ollama.embeddings(model='jina/jina-embeddings-v2-base-de', prompt='Wie ist das Wetter heute?')

print(len(res['embedding']))

In [ ]:
from utils import call_jina_reader

res = call_jina_reader("https://en.wikipedia.org/wiki/New_England")

print(res)

# 测试云数据库

In [ ]:
from utils import db_insert, db_search
from data_prepare import proc_one_url

proc_one_url("https://en.wikipedia.org/wiki/New_England", "wiki")

db_search("similarities between the historical administrative divisions of New England in the United States and Australia", "wiki")

# 测试整体插入

In [ ]:
from data_prepare import gather_urls

gather_urls('FRAMES.csv')

In [ ]:
from data_prepare import proc_n_urls

proc_n_urls("urls.txt", "wiki")

# 测试dataset采样

In [ ]:
import pandas as pd

df = pd.read_csv("FRAMES.csv")

# 随机抽取50行

df_sample = df.sample(n=50, random_state=42)

df_sample.to_csv("FRAMES_50.csv", index=False)

# 获取LLM生成结果

In [ ]:
from exp import *

for start in range(0, 801, 50):
    get_pred("FRAMES.csv", "deepseek-r1-0528", start, start+50)

# test eval

In [ ]:
from exp import proc_json, eval_ans

raw_jsons = ["output/gpt-4o_answers.json", "output/deepseek-r1-0528_answers.json", "output/qwen3-32b_answers.json"] 


# eval_ans("1", "1", "gpt-4o")
proc_json(raw_jsons[1])
proc_json(raw_jsons[2])

In [ ]:
import re

string = f"""
> search("\n        You are an AI assistant. Please carefully analyze the following question and provide the answer.  \n        Important: Only output the answer itself, and wrap your answer with <ans></ans> tags.  \n        Do not explain anything. Do not output anything else beyond the answer in the tags.\n\n        Question: According to topographical summit prominence, how many years were there between the first ascent of the United State's second most prominent mountain and the first ascent of Russia's second most prominent mountain? \n    ")

<ans>10</ans>
"""

string = re.sub(r'> search\((.*?)\)', '', string, flags=re.DOTALL)

print(string)

# final_ans = re.search(r'<ans>(.*?)</ans>', string).group(1).strip()

In [ ]:
import json

json_paths = ["evaloutput/gpt-4o_answers.json", "evaloutput/deepseek-r1-0528_answers.json", "evaloutput/qwen3-32b_answers.json"]

def acc_all(json_paths):
    for json_path in json_paths:
        with open(json_path, "r") as f:
            data = json.load(f)
            acc_count = 0
            total_count = len(data)
            for item in data:
                if item['is_correct'] == "true":
                    acc_count += 1
            acc = acc_count / total_count if total_count > 0 else 0
            print(f"Accuracy for {json_path}: {acc:.2%}")
acc_all(json_paths)
